In [1]:
#Импортируем необходимыме библиотеки
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
image_size = (180, 180) #Размер изображение, нужно подбирать рукаи
batch_size = 64 #Размер батча зависит от того сколько влазит в видеокарту/оперативную память
epochs = 5 #Количество эпох на обучение

In [3]:
#Создание train generator

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=image_size,
    batch_size=batch_size    
)

Found 20000 images belonging to 2 classes.


In [4]:
#Создание validation generator
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

validation_generator = validation_datagen.flow_from_directory(
    'data/valid',
    target_size=image_size,
    batch_size=batch_size,
)

Found 5000 images belonging to 2 classes.


In [5]:
# Модель для обучения
def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    
    x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    previous_block_activation = x 

    for size in [128, 256, 512, 728]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

        residual = layers.Conv2D(size, 1, strides=2, padding='same')(
            previous_block_activation
        )
        x = layers.add([x, residual]) 
        previous_block_activation = x 

    x = layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(2, activation='softmax')(x)
    
    return keras.Model(inputs, outputs)

In [6]:
model = make_model(input_shape=image_size + (3,))

In [7]:
#Компеляция модели для обучения
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 180, 180, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 90, 90, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 90, 90, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 90, 90, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [9]:
#Сохранение весов в процессе обучения
callbacks = [
    keras.callbacks.ModelCheckpoint('save/{epoch}.h5'),
]

In [10]:
model.fit_generator(    
        train_generator,
        steps_per_epoch=train_generator.n//batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.n//batch_size,
        callbacks=callbacks
)

C:\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
312/312 [==============================] - 177s 533ms/step - loss: 0.7099 - accuracy: 0.6305 - val_loss: 0.8634 - val_accuracy: 0.5002
Epoch 2/5
312/312 [==============================] - 138s 443ms/step - loss: 0.4153 - accuracy: 0.8158 - val_loss: 0.4481 - val_accuracy: 0.8153
Epoch 3/5
312/312 [==============================] - 146s 466ms/step - loss: 0.2652 - accuracy: 0.8902 - val_loss: 0.3507 - val_accuracy: 0.8472
Epoch 4/5
312/312 [==============================] - 141s 451ms/step - loss: 0.1956 - accuracy: 0.9191 - val_loss: 0.3932 - val_accuracy: 0.8452
Epoch 5/5
312/312 [==============================] - 144s 461ms/step - loss: 0.1281 - accuracy: 0.9487 - val_loss: 0.2882 - val_accuracy: 0.8816
